In [1]:
import sys, os

In [2]:
sys.path.append(os.path.abspath(os.path.join('../scripts')))

In [4]:

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import CohereEmbeddings
from langchain_community.vectorstores import FAISS

from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank, FlashrankRerank
from langchain_community.llms import Cohere
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_experimental.text_splitter import SemanticChunker
from langchain.vectorstores import Chroma
from langchain.document_loaders import Docx2txtLoader



In [5]:
from RAG_utils import load_document, load_documents_from_folder

/home/elias/miniconda/envs/langc/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [26]:
import getpass
import os

os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

In [6]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [7]:
folder_path = "../data/Evaluation Sets/contract"  # Replace with your folder path
all_documents = load_documents_from_folder(folder_path)

In [ ]:
all_documents

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=50)
split_texts = text_splitter.split_documents(all_documents)

In [9]:
len(split_texts)

466

In [10]:
vectordb=FAISS.from_documents(split_texts, OpenAIEmbeddings())

/home/elias/miniconda/envs/langc/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [13]:
# retriever = FAISS.from_documents(split_texts, OpenAIEmbeddings()).as_retriever(
#     search_kwargs={"k": 5}
# )

/home/elias/miniconda/envs/langc/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [11]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

In [27]:
retriever = vectordb.as_retriever()

In [28]:
compressor = CohereRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

In [29]:
llm = ChatOpenAI(temperature=0)
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=compression_retriever, llm=llm
)

In [13]:
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [14]:
question = "How much is the escrow amount?"

In [30]:
unique_docs = retriever_from_llm.get_relevant_documents(query=question)
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Can you explain the process of issuing a termination notice?', '2. How is a termination notice typically handled in legal agreements?', '3. What are the key components of a termination notice and how is it delivered to the relevant parties?']


9

In [16]:
unique_docs

[Document(page_content='Escrow.', metadata={'source': '../data/Evaluation Sets/contract/Raptor Contract.docx'}),
 Document(page_content='“Equity Interest” means, with respect to any Person, (a)\xa0any capital stock, partnership or membership interest, unit of participation or other similar interest (however designated) in such Person and (b)\xa0any option, warrant, purchase right, conversion right, exchange right or other Contractual Obligation which would entitle any other Person to acquire any such interest in such Person or otherwise entitle any other Person to share in the equity, profits, earnings, losses or gains of such Person (including stock appreciation, phantom stock, profit participation or other similar rights).\n\n“Escrow Amount” means, $1,000,000', metadata={'source': '../data/Evaluation Sets/contract/Raptor Contract.docx'}),
 Document(page_content='“Escrow Amount” means, $1,000,000\n\n “Facilities” means any buildings, plants, improvements or structures located on the R

In [18]:
chain = RetrievalQA.from_chain_type(
    llm=OpenAI(temperature=0), retriever=retriever_from_llm
)

In [41]:
question=" Is the Advisor entitled to social benefits? "

In [42]:
chain({"query": question})

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What social benefits is the Advisor entitled to?', '2. Are there any social benefits that the Advisor can claim?', '3. Can the Advisor receive any social benefits?']


{'query': ' Is the Advisor entitled to social benefits? ',
 'result': ' No, the Advisor is not entitled to social benefits as they are an independent contractor and not an employee of the Company.'}